In [1]:
import pandas as pd

base_path = 'C:\\Users\\zxy\\Desktop\\Capstone\\Data\\'
Ricoh_Opps = pd.read_excel(f'{base_path}Copy of Ricoh_Opps_08022024.xlsx')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import statsmodels.api as sm
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')


In [6]:
opp = Ricoh_Opps
opp.columns = opp.columns.str.lower()

In [7]:
#prop of closed deal
opp[opp.closed == 1].shape[0]/opp.shape[0]

0.8843518143142532

In [8]:
# how many deals not closed but have order rev?
opp[(opp.closed == 0) & (opp.order_rev.notnull()) & (opp.order_rev!=0)].shape[0]

239

In [9]:
# % NA order_rev
opp[opp.order_rev.isnull()].shape[0]/opp.shape[0]

0.8309029321115795

In [10]:
# % NA order_rev in closed deals
opp_closed = opp[opp.closed == 1]
opp_closed[opp_closed.order_rev.isnull()].shape[0]/opp_closed.shape[0]

0.8092026776246023

## Ricoh_Opps 

In [11]:
opp.isna().mean()*100

key                    0.000000
lead_number           83.186516
opportunity_number    14.344292
quote_number          41.374864
order_number          63.885646
                        ...    
nps                   86.959967
department            62.371479
job_type              60.176303
employees_dnb          9.113333
sales_dnb              9.113333
Length: 73, dtype: float64

In [12]:
# drop irrelavant cols
opp = opp.drop(['quote_number', 'order_number'], axis = 1)

In [ ]:
# L2 is missing

## Lead 

In [13]:
opp.groupby('has_lead').apply(lambda x: x.isna().mean()*100)[['lead_number', 'opportunity_number']]

,lead_number,opportunity_number
has_lead,,
0,100.0,0.000000
1,0.0,85.314218


In [14]:
lead_cols = [col for col in opp.columns if 'lead' in col]
opp_lead = opp[opp.lead_number.notnull()][lead_cols]
opp_lead.isna().mean()*100

lead_number              0.000000
lead_status_name         0.000000
lead_sub_source          8.324437
lead_primary_product     5.316252
lead_date                0.000000
has_lead                 0.000000
lead                    54.775548
dtype: float64

In [15]:
opp_lead.groupby('lead_sub_source').size().sort_values(ascending = False).head(10)

lead_sub_source
Website_RUSA             22545
Paid Search              12360
Webinar                  11353
RUSA_Sales_Contact_Us     7822
RUSA_Request_A_Quote      7114
Cost Per Lead             6940
Content Syndication       6303
Paid Leads                5687
Inbound                   5003
Website_RUSA_v2           4044
dtype: int64

In [16]:
# create a Unknown category for missing in source, primary
opp[['lead_primary_product', 'lead_sub_source']].fillna('Unknown', inplace=True)

In [17]:
opp.head()

,key,lead_number,opportunity_number,l2_rise_number,funnel_stage,lead_status_name,lead_sub_source,lead_primary_product,customer_segment,crm_industry,...,svc_calls,cv,mif,renewal,sat,nps,department,job_type,employees_dnb,sales_dnb
0,O-1001591177,NaN,O-1001591177,A-1000882103,SQL,NaN,NaN,NaN,Federal,Federal,...,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,Unknown,1965.0,2.346090e+06
1,O-1000558710,NaN,O-1000558710,A-0013759349,SQL,NaN,NaN,NaN,Enterprise,Healthcare,...,2233.0,144586274.0,1670.0,0.0,NaN,NaN,Operations / Compliance / Reco,Director,1683.0,2.579766e+09
2,O-1002059945,NaN,O-1002059945,A-0013508952,SQL,NaN,NaN,NaN,Strategic,Banking & Finance,...,3210.0,719873594.0,26046.0,1.0,NaN,NaN,Procurement / Purchasing,Consultant,275502.0,8.529614e+10
3,O-1002269225,NaN,O-1002269225,A-1000882135,SQL,NaN,NaN,NaN,Federal,Federal,...,3501.0,379055914.0,2485.0,1105.0,8.67,8.33,Administration,Manager / Supervisor,230.0,3.033240e+07
4,O-1001952436,NaN,O-1001952436,A-0013508952,SQL,NaN,NaN,NaN,Strategic,Banking & Finance,...,3089.0,725387056.0,26098.0,2.0,NaN,NaN,NaN,Director,275502.0,8.529614e+10


## Flag 

In [18]:
flag_cols = [col for col in opp.columns if 'flag' in col]
flag_cols.remove('mfp_flag')
print(flag_cols)
flag_na = opp[opp[flag_cols].isna().all(axis = 1)]
flag_na.isna().mean()*100

['a4_bw_flag', 'a4_c_flag', 'a3_bw_flag', 'a3_c_flag', 'docuware_flag', 'ransomcare_flag', 'it_health_check_flag', 'ms_flag', 'pp_bw_cs_flag', 'pp_c_cs_flag', 'wf_bw_flag', 'wf_c_flag', 'bw_lp_flag', 'color_lp_flag', 'mindshift_flag']


key                     0.000000
lead_number             0.000000
opportunity_number    100.000000
l2_rise_number         62.421895
funnel_stage            0.000000
                         ...    
nps                    96.787265
department             78.792883
job_type               77.904997
employees_dnb          62.429556
sales_dnb              62.429556
Length: 71, dtype: float64

## Check NA 

In [19]:
# Check for missing values in each column
na_columns = opp.isna().sum()


In [20]:
# Filter to show only columns with missing values
columns_with_na = na_columns[na_columns > 0]

if columns_with_na.empty:
    print("No columns have missing values.")
else:
    print("Columns with missing values:")
    print(columns_with_na)

Columns with missing values:
lead_number             681241
opportunity_number      117470
l2_rise_number           74589
lead_status_name        681241
lead_sub_source         692703
lead_primary_product    688561
a4_bw_flag              117470
a4_c_flag               117470
a3_bw_flag              117470
a3_c_flag               117470
docuware_flag           117470
ransomcare_flag         117470
it_health_check_flag    117470
ms_flag                 117470
pp_bw_cs_flag           117470
pp_c_cs_flag            117470
wf_bw_flag              117470
wf_c_flag               117470
bw_lp_flag              117470
color_lp_flag           117470
mindshift_flag          117470
lead_date               681241
opp_date                117470
opp_rev                 118382
quote_date              338832
quote_rev               338832
order_date              680341
order_rev               680453
close_date              201761
sales_stage_status      117470
close_days              201761
eloqua    

## Categorical data 

In [21]:
# Get categorical columns (including string/object data types)
categorical_columns = opp.select_dtypes(include=['object', 'category']).columns

# Get numerical columns (including int and float data types)
numerical_columns = opp.select_dtypes(include=['number']).columns

print("Categorical columns:")
print(categorical_columns)

print("\nNumerical columns:")
print(numerical_columns)

Categorical columns:
Index(['key', 'lead_number', 'opportunity_number', 'l2_rise_number',
       'funnel_stage', 'lead_status_name', 'lead_sub_source',
       'lead_primary_product', 'customer_segment', 'crm_industry',
       'sales_stage_status', 'department', 'job_type'],
      dtype='object')

Numerical columns:
Index(['a4_bw_flag', 'a4_c_flag', 'a3_bw_flag', 'a3_c_flag', 'docuware_flag',
       'ransomcare_flag', 'it_health_check_flag', 'ms_flag', 'pp_bw_cs_flag',
       'pp_c_cs_flag', 'wf_bw_flag', 'wf_c_flag', 'bw_lp_flag',
       'color_lp_flag', 'mfp_flag', 'mindshift_flag', 'opp_rev', 'quote_rev',
       'order_rev', 'closed', 'has_lead', 'open_days', 'close_days', 'win',
       'rev', 'eloqua', 'eloqua oc', 'lead', 'opp', 'order', 'quote', 'sales',
       'web', 'webinar', 'cs', 'hw', 'its', 'ms off', 'ms on', 'ps', 'refi',
       'rent', 'supp', 'ts', 'svc_calls', 'cv', 'mif', 'renewal', 'sat', 'nps',
       'employees_dnb', 'sales_dnb'],
      dtype='object')


In [22]:
opp[['lead_primary_product', 'lead_sub_source','lead_status_name','lead_number']].fillna('Unknown', inplace=True)

## numerical data 

In [23]:
# For numerical columns using median
opp['opp_rev'].fillna(opp['opp_rev'].median(), inplace=True)
opp['quote_rev'].fillna(opp['quote_rev'].median(), inplace=True)
opp['order_rev'].fillna(opp['order_rev'].median(), inplace=True)
opp['rev'].fillna(opp['rev'].median(), inplace=True)
opp['eloqua'].fillna(opp['eloqua'].median(), inplace=True)
opp['eloqua oc'].fillna(opp['eloqua oc'].median(), inplace=True)
opp['lead'].fillna(opp['lead'].median(), inplace=True)
opp['opp'].fillna(opp['opp'].median(), inplace=True)
opp['order'].fillna(opp['order'].median(), inplace=True)
opp['quote'].fillna(opp['quote'].median(), inplace=True)
opp['sales'].fillna(opp['sales'].median(), inplace=True)
opp['sat'].fillna(opp['sat'].median(), inplace=True)
opp['nps'].fillna(opp['nps'].median(), inplace=True)


## Binary data

In [24]:
# For binary flags, assuming missing values mean 'False'
opp['a4_bw_flag'].fillna(0, inplace=True)
opp['a4_c_flag'].fillna(0, inplace=True)
opp['a3_bw_flag'].fillna(0, inplace=True)
opp['a3_c_flag'].fillna(0, inplace=True)
opp['docuware_flag'].fillna(0, inplace=True)
opp['ransomcare_flag'].fillna(0, inplace=True)
opp['it_health_check_flag'].fillna(0, inplace=True)
opp['ms_flag'].fillna(0, inplace=True)
opp['pp_bw_cs_flag'].fillna(0, inplace=True)
opp['pp_bw_cs_flag'].fillna(0, inplace=True)
opp['pp_c_cs_flag'].fillna(0, inplace=True)
opp['wf_bw_flag'].fillna(0, inplace=True)
opp['wf_c_flag'].fillna(0, inplace=True)
opp['bw_lp_flag'].fillna(0, inplace=True)
opp['color_lp_flag'].fillna(0, inplace=True)
opp['mfp_flag'].fillna(0, inplace=True)
opp['mindshift_flag'].fillna(0, inplace=True)



## Date columns 

In [25]:
# For date columns, filling with NaT (missing date)
#If missing dates are due to a lack of an event (e.g., no order placed), filling with a placeholder like NaT (Not a Time) can work.
opp['lead_date'].fillna(pd.NaT, inplace=True)
opp['opp_date'].fillna(pd.NaT, inplace=True)
opp['quote_date'].fillna(pd.NaT, inplace=True)
opp['order_date'].fillna(pd.NaT, inplace=True)
opp['close_date'].fillna(pd.NaT, inplace=True)
opp['close_days'].fillna(pd.NaT, inplace=True)


## Performance metrics columns 

In [35]:
# For columns with performance metrics, filling with 0
#assuming they represent no activity.
opp['eloqua'].fillna(0, inplace=True)
opp['eloqua oc'].fillna(0, inplace=True)
opp['lead'].fillna(0, inplace=True)
opp['opp'].fillna(0, inplace=True)
opp['order'].fillna(0, inplace=True)
opp['quote'].fillna(0, inplace=True)
opp['sales'].fillna(0, inplace=True)
opp['web'].fillna(0, inplace=True)
opp['webinar'].fillna(0, inplace=True)
opp[['cs', 'hw', 'its', 'ms off', 'ms on', 'ps', 'refi', 'rent', 'supp', 'ts', 
     'svc_calls', 'cv', 'mif', 'renewal', 'department', 'job_type']].fillna(0, inplace=True)
opp['employees_dnb'].fillna(0, inplace=True)
opp['sales_dnb'].fillna(0, inplace=True)
opp['sales_stage_status'].fillna(0, inplace=True)


## Now check NA again 

In [36]:
# Temporarily increase the number of rows displayed
pd.set_option('display.max_rows', None)

# Check missing values again
print(opp.isna().sum())

key                          0
lead_number             681241
opportunity_number      117470
l2_rise_number           74589
funnel_stage                 0
lead_status_name        681241
lead_sub_source         692703
lead_primary_product    688561
customer_segment             0
crm_industry                 0
a4_bw_flag                   0
a4_c_flag                    0
a3_bw_flag                   0
a3_c_flag                    0
docuware_flag                0
ransomcare_flag              0
it_health_check_flag         0
ms_flag                      0
pp_bw_cs_flag                0
pp_c_cs_flag                 0
wf_bw_flag                   0
wf_c_flag                    0
bw_lp_flag                   0
color_lp_flag                0
mfp_flag                     0
mindshift_flag               0
lead_date               681241
opp_date                117470
opp_rev                      0
quote_date              338832
quote_rev                    0
order_date              680341
order_re

In [38]:
opp.head()

,key,lead_number,opportunity_number,l2_rise_number,funnel_stage,lead_status_name,lead_sub_source,lead_primary_product,customer_segment,crm_industry,...,svc_calls,cv,mif,renewal,sat,nps,department,job_type,employees_dnb,sales_dnb
0,O-1001591177,NaN,O-1001591177,A-1000882103,SQL,NaN,NaN,NaN,Federal,Federal,...,NaN,NaN,NaN,NaN,7.60,7.75,Unknown,Unknown,1965.0,2.346090e+06
1,O-1000558710,NaN,O-1000558710,A-0013759349,SQL,NaN,NaN,NaN,Enterprise,Healthcare,...,2233.0,144586274.0,1670.0,0.0,7.60,7.75,Operations / Compliance / Reco,Director,1683.0,2.579766e+09
2,O-1002059945,NaN,O-1002059945,A-0013508952,SQL,NaN,NaN,NaN,Strategic,Banking & Finance,...,3210.0,719873594.0,26046.0,1.0,7.60,7.75,Procurement / Purchasing,Consultant,275502.0,8.529614e+10
3,O-1002269225,NaN,O-1002269225,A-1000882135,SQL,NaN,NaN,NaN,Federal,Federal,...,3501.0,379055914.0,2485.0,1105.0,8.67,8.33,Administration,Manager / Supervisor,230.0,3.033240e+07
4,O-1001952436,NaN,O-1001952436,A-0013508952,SQL,NaN,NaN,NaN,Strategic,Banking & Finance,...,3089.0,725387056.0,26098.0,2.0,7.60,7.75,NaN,Director,275502.0,8.529614e+10


In [39]:
print(opp[['cs', 'hw', 'its', 'ms off', 'ms on', 'ps', 'refi', 'rent', 'supp', 'ts', 'svc_calls', 'cv', 'mif', 'renewal', 'department', 'job_type']].dtypes)


cs            float64
hw            float64
its           float64
ms off        float64
ms on         float64
ps            float64
refi          float64
rent          float64
supp          float64
ts            float64
svc_calls     float64
cv            float64
mif           float64
renewal       float64
department     object
job_type       object
dtype: object


In [41]:
#inplace=True is not working as expected, try assigning the result of fillna back to the DataFrame:
opp[['cs', 'hw', 'its', 'ms off', 'ms on', 'ps', 'refi', 'rent', 'supp', 'ts', 'svc_calls', 'cv', 'mif', 'renewal']] = opp[['cs', 'hw', 'its', 'ms off', 'ms on', 'ps', 'refi', 'rent', 'supp', 'ts', 'svc_calls', 'cv', 'mif', 'renewal']].fillna(0)


In [42]:
print(opp[['cs', 'hw', 'its', 'ms off', 'ms on', 'ps', 'refi', 'rent', 'supp', 'ts', 'svc_calls', 'cv', 'mif', 'renewal']].isna().sum())


cs           0
hw           0
its          0
ms off       0
ms on        0
ps           0
refi         0
rent         0
supp         0
ts           0
svc_calls    0
cv           0
mif          0
renewal      0
dtype: int64


In [43]:
opp['department'].fillna('Unknown', inplace=True)  
opp['job_type'].fillna('Unknown', inplace=True)


In [44]:
print(opp[['department', 'job_type']].isna().sum())


department    0
job_type      0
dtype: int64


## Try another way to impute date columns NA but seems not working

In [47]:
print(opp[['lead_date', 'opp_date', 'quote_date', 'order_date', 'close_date', 'close_days']].dtypes)


lead_date     datetime64[ns]
opp_date      datetime64[ns]
quote_date    datetime64[ns]
order_date    datetime64[ns]
close_date    datetime64[ns]
close_days            object
dtype: object


In [51]:
opp['close_days'].fillna('N/A', inplace=True)

In [52]:
print(opp['close_days'].isna().sum())

0


In [56]:
opp['lead_date'] = pd.to_datetime(opp['lead_date'], errors='coerce')
opp['opp_date'] = pd.to_datetime(opp['opp_date'], errors='coerce')
opp['quote_date'] = pd.to_datetime(opp['quote_date'], errors='coerce')
opp['order_date'] = pd.to_datetime(opp['order_date'], errors='coerce')
opp['close_date'] = pd.to_datetime(opp['close_date'], errors='coerce')


In [57]:
opp['lead_date'].fillna(pd.NaT, inplace=True)
opp['opp_date'].fillna(pd.NaT, inplace=True)
opp['quote_date'].fillna(pd.NaT, inplace=True)
opp['order_date'].fillna(pd.NaT, inplace=True)
opp['close_date'].fillna(pd.NaT, inplace=True)


In [58]:
print(opp[['lead_date', 'opp_date', 'quote_date', 'order_date', 'close_date']].isna().sum())


lead_date     681241
opp_date      117470
quote_date    338832
order_date    680341
close_date    201761
dtype: int64


In [59]:
print("Unique values in lead_date:")
print(opp['lead_date'].unique())

print("Unique values in opp_date:")
print(opp['opp_date'].unique())

print("Unique values in quote_date:")
print(opp['quote_date'].unique())

print("Unique values in order_date:")
print(opp['order_date'].unique())

print("Unique values in close_date:")
print(opp['close_date'].unique())


Unique values in lead_date:
<DatetimeArray>
[                'NaT', '2021-09-13 00:00:00', '2021-05-14 00:00:00',
 '2021-06-07 00:00:00', '2022-05-04 00:00:00', '2020-01-15 00:00:00',
 '2019-11-20 00:00:00', '2023-10-25 00:00:00', '2023-05-23 00:00:00',
 '2019-06-27 00:00:00',
 ...
 '2020-04-12 00:00:00', '2023-07-01 00:00:00', '2021-06-20 00:00:00',
 '2022-03-13 00:00:00', '2022-04-23 00:00:00', '2022-03-19 00:00:00',
 '2021-12-25 00:00:00', '2024-04-07 00:00:00', '2022-06-26 00:00:00',
 '2018-03-09 00:00:00']
Length: 2059, dtype: datetime64[ns]
Unique values in opp_date:
<DatetimeArray>
['2021-09-08 00:00:00', '2019-02-28 00:00:00', '2022-09-16 00:00:00',
 '2023-03-18 00:00:00', '2022-07-12 00:00:00', '2019-11-18 00:00:00',
 '2020-11-30 00:00:00', '2021-11-02 00:00:00', '2021-12-01 00:00:00',
 '2022-06-23 00:00:00',
 ...
 '2022-11-26 00:00:00', '2022-07-03 00:00:00', '2018-08-23 00:00:00',
 '2018-01-16 00:00:00', '2024-07-21 00:00:00', '2018-07-27 00:00:00',
 '2020-04-19 00:00:00',  

In [60]:
# Temporarily increase the number of rows displayed
pd.set_option('display.max_rows', None)

# Check missing values again
print(opp.isna().sum())

key                          0
lead_number             681241
opportunity_number      117470
l2_rise_number           74589
funnel_stage                 0
lead_status_name        681241
lead_sub_source         692703
lead_primary_product    688561
customer_segment             0
crm_industry                 0
a4_bw_flag                   0
a4_c_flag                    0
a3_bw_flag                   0
a3_c_flag                    0
docuware_flag                0
ransomcare_flag              0
it_health_check_flag         0
ms_flag                      0
pp_bw_cs_flag                0
pp_c_cs_flag                 0
wf_bw_flag                   0
wf_c_flag                    0
bw_lp_flag                   0
color_lp_flag                0
mfp_flag                     0
mindshift_flag               0
lead_date               681241
opp_date                117470
opp_rev                      0
quote_date              338832
quote_rev                    0
order_date              680341
order_re